In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from os import getcwd, listdir
from os.path import join, isfile, abspath, pardir
from time import time
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from shutil import copyfile

In [3]:
conn_str = "jdbc:mysql://localhost/test_tpcds"

In [4]:
spark = SparkSession.builder.master("local[1]").appName("JAS").enableHiveSupport().getOrCreate()

In [5]:
tables = ["call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "dbgen_version", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site"]

In [6]:
schema_dict = {}
df_dict = {}

for index, table in enumerate(tables):
    # print("{}: '{}' table\n".format( index + 1, table ))
    table_df = spark.read.format("jdbc").option("driver", "com.mysql.jdbc.Driver").option("url", conn_str).option("dbtable", table).option("user", "root").load()
    schema_dict[table] = table_df.schema
    df = spark.read.load( join(getcwd(), "data_correct", "{}.csv".format( table )),
        format="csv",
        delimiter="|",
        header="true",
        schema=table_df.schema)
    df_dict[table] = df
    df.createOrReplaceTempView("{}".format( table ))

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
21/10/22 16:32:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
project_dir = getcwd()
sql_queries_path = join(project_dir, "correct_sql_queries")
# sql_queries_path = join(getcwd(), "correct_sql_queries")

# project_dir = join(abspath(join(getcwd(), pardir)), "data-warehouse-project")
# sql_queries_path = join(project_dir, "sql_queries")

print("Total {} queries in '{}'".format(len(listdir(sql_queries_path)), sql_queries_path))

Total 100 queries in '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/correct_sql_queries'


In [ ]:
# for query_file in listdir(sql_queries_path):
#     file_path = join(sql_queries_path, query_file)
#     if not isfile(file_path): continue
#     with open(file_path, "rt") as file:
#         sql_code = file.read()
#         spark.sql(sql_code).show()

In [12]:
spark.sql("""
-- start query 1 in stream 0 using template query39.tpl
with inv as
(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
       ,stdev,mean, case mean when 0 then null else stdev/mean end cov
 from(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
            ,stddev_samp(inv_quantity_on_hand) stdev,avg(inv_quantity_on_hand) mean
      from inventory
          ,item
          ,warehouse
          ,date_dim
      where inv_item_sk = i_item_sk
        and inv_warehouse_sk = w_warehouse_sk
        and inv_date_sk = d_date_sk
        and d_year =2001
      group by w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy) foo
 where case mean when 0 then 0 else stdev/mean end > 1)
select inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean, inv1.cov
        ,inv2.w_warehouse_sk,inv2.i_item_sk,inv2.d_moy,inv2.mean, inv2.cov
from inv inv1,inv inv2
where inv1.i_item_sk = inv2.i_item_sk
  and inv1.w_warehouse_sk =  inv2.w_warehouse_sk
  and inv1.d_moy=1
  and inv2.d_moy=1+1
order by inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean,inv1.cov
        ,inv2.d_moy,inv2.mean, inv2.cov
;
with inv as
(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
       ,stdev,mean, case mean when 0 then null else stdev/mean end cov
 from(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
            ,stddev_samp(inv_quantity_on_hand) stdev,avg(inv_quantity_on_hand) mean
      from inventory
          ,item
          ,warehouse
          ,date_dim
      where inv_item_sk = i_item_sk
        and inv_warehouse_sk = w_warehouse_sk
        and inv_date_sk = d_date_sk
        and d_year =2001
      group by w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy) foo
 where case mean when 0 then 0 else stdev/mean end > 1)
select inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean, inv1.cov
        ,inv2.w_warehouse_sk,inv2.i_item_sk,inv2.d_moy,inv2.mean, inv2.cov
from inv inv1,inv inv2
where inv1.i_item_sk = inv2.i_item_sk
  and inv1.w_warehouse_sk =  inv2.w_warehouse_sk
  and inv1.d_moy=1
  and inv2.d_moy=1+1
  and inv1.cov > 1.5
order by inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean,inv1.cov
        ,inv2.d_moy,inv2.mean, inv2.cov
;

-- end query 1 in stream 0 using template query39.tpl
""")

ParseException: 
extraneous input 'with' expecting {<EOF>, ';'}(line 28, pos 0)

== SQL ==

-- start query 1 in stream 0 using template query39.tpl
with inv as
(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
       ,stdev,mean, case mean when 0 then null else stdev/mean end cov
 from(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
            ,stddev_samp(inv_quantity_on_hand) stdev,avg(inv_quantity_on_hand) mean
      from inventory
          ,item
          ,warehouse
          ,date_dim
      where inv_item_sk = i_item_sk
        and inv_warehouse_sk = w_warehouse_sk
        and inv_date_sk = d_date_sk
        and d_year =2001
      group by w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy) foo
 where case mean when 0 then 0 else stdev/mean end > 1)
select inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean, inv1.cov
        ,inv2.w_warehouse_sk,inv2.i_item_sk,inv2.d_moy,inv2.mean, inv2.cov
from inv inv1,inv inv2
where inv1.i_item_sk = inv2.i_item_sk
  and inv1.w_w

##### Make Separate files

In [ ]:
with open(join(sql_queries_path, "tpcds_queries.sql"), "rt") as file:
    lines = file.readlines()
    query = ""
    query_no = 1
    for index, line in enumerate(lines):
        query_line = line.strip()
        query += "\n{}".format( query_line )
        
        # Separate each query
        if query_line.find("-- end query", 0, len(query_line)) != -1:
            
            with open(join(sql_queries_path, "query_{}.sql".format(query_no)), "w") as query_file:
                query_file.write(query)
                query = ""
                print("Query - {} written.".format(query_no))
                query_no = query_no + 1

##### Do the benchmarking

In [8]:
take_avg_run, skip_run = 5, 1
total_runs = take_avg_run + skip_run

In [9]:
def get_execute_query_time(file, debug=False):
    with open(file, "rt") as f:
        try:
            start_time = time()
            spark.sql(f.read())
            return time() - start_time      
        except Exception as e:
            print("[[ error ]]: Failed to execute '{}' query :(".format( file ))
            if debug:
                print("[[ error ]]:\n{}\n".format( e ))
            return None

In [10]:
def run_queries(queries_list, queries_path, debug=False):
    timings_dict = {}
    for index, query in enumerate(queries_list):
        print("="*50)
        print("{} - Running benchmark for query: {}".format(index + 1, query))

        timings = list()
        query_file = join(queries_path, query)

        for run in list(range(0, total_runs)):
            executed_time = get_execute_query_time(query_file, debug)
            if executed_time is None: continue
            if run == 0:
                print("Skipping 1st run")
            else:
                timings.append(executed_time)
                print("{} run took {} secs".format(run, executed_time))
        timings_dict[query] = timings
    return timings_dict

In [11]:
timings_dict = {}
total_queries = sorted([x for x in listdir(sql_queries_path) if str(x).startswith("query_")])
timings_dict = run_queries(queries_list = total_queries, queries_path = sql_queries_path, debug=False)

1 - Running benchmark for query: query_1.sql
Skipping 1st run
1 run took 0.011373043060302734 secs
2 run took 0.012604951858520508 secs
3 run took 0.01088094711303711 secs
4 run took 0.008837223052978516 secs
5 run took 0.008102893829345703 secs
2 - Running benchmark for query: query_10.sql
Skipping 1st run
1 run took 0.03395700454711914 secs
2 run took 0.037383079528808594 secs
3 run took 0.04610610008239746 secs
4 run took 0.03131580352783203 secs
5 run took 0.027215242385864258 secs
3 - Running benchmark for query: query_11.sql
Skipping 1st run
1 run took 0.020309925079345703 secs
2 run took 0.017882108688354492 secs
3 run took 0.017178773880004883 secs
4 run took 0.01779913902282715 secs
5 run took 0.015356779098510742 secs
4 - Running benchmark for query: query_12.sql
Skipping 1st run
1 run took 0.010740041732788086 secs
2 run took 0.012111186981201172 secs
3 run took 0.010136842727661133 secs
4 run took 0.01304006576538086 secs
5 run took 0.011892080307006836 secs
5 - Running ben

In [ ]:
i = int(0)
for key, value in timings_dict.items():
    if len(value):
        i = i + 1
        print("="*20)
        print("\n{} - {}: {} secs\n".format(i, key, sum(value)/len(value)))
        # print("Avg.: {}".format( sum(value)/len(value) ))

###### Failed Queries

In [ ]:
failed_queries = [x for x in list(timings_dict.keys()) if not len(timings_dict[x])]
failed_queries

In [ ]:
modified_queries_path = join(project_dir, "modified_queries")

In [ ]:
total_queries = 4
for index, query in enumerate(failed_queries):
    if total_queries == index:
        break
    file_name = query.split('.')[0]
    copyfile(join(sql_queries_path, query), join(modified_queries_path, "{}_original.sql".format(file_name)))
    copyfile(join(sql_queries_path, query), join(modified_queries_path, "{}_modified.sql".format(file_name)))

###### Run the modified queries

In [ ]:
modified_queries = [x for x in listdir(modified_queries_path) if x.endswith("_modified.sql")]
for index, query in enumerate(modified_queries):
    print("{} - {}".format(index + 1, query))

In [ ]:
timings_dict = run_queries(queries_list = modified_queries, queries_path = modified_queries_path, debug=True)